# NeSy-Core Quickstart

This notebook walks through the basics of NeSy-Core:
1. Creating a model
2. Adding symbolic rules
3. Adding concept graph edges
4. Running reasoning
5. Interpreting the output

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))

from nesy.api.nesy_model import NeSyModel
from nesy.core.types import Predicate, SymbolicRule, ConceptEdge

## 1. Create a Model

Choose a domain (medical, legal, code, general) and set strict mode.

In [ ]:
model = NeSyModel(domain="medical", strict_mode=True)
print(f"Domain: medical | Rules: {model.rule_count}")

## 2. Add Symbolic Rules

Rules encode domain knowledge as IF-THEN statements with confidence weights.

In [ ]:
model.add_rules([
    SymbolicRule(
        id="fever_flu",
        antecedents=[Predicate("fever"), Predicate("cough")],
        consequents=[Predicate("flu_likely")],
        weight=0.85,
        domain="medical",
    ),
    SymbolicRule(
        id="flu_rest",
        antecedents=[Predicate("flu_likely")],
        consequents=[Predicate("recommend_rest")],
        weight=0.90,
        domain="medical",
    ),
])
print(f"Rules loaded: {model.rule_count}")

## 3. Add Concept Graph Edges (NSI World Model)

Edges define co-occurrence, causal, and temporal relationships between concepts.

In [ ]:
model.add_concept_edges([
    ConceptEdge(source="fever", target="cough",
                cooccurrence_prob=0.7, causal_strength=0.5, temporal_stability=1.0),
    ConceptEdge(source="fever", target="headache",
                cooccurrence_prob=0.6, causal_strength=0.5, temporal_stability=1.0),
    ConceptEdge(source="cough", target="sore_throat",
                cooccurrence_prob=0.5, causal_strength=0.5, temporal_stability=1.0),
])
print(f"Graph stats: {model.concept_graph_stats}")

## 4. Run Reasoning

Provide observed facts and get an NSIOutput with answer, confidence, and null set.

In [ ]:
output = model.reason(
    facts={Predicate("fever"), Predicate("cough")},
    context_type="medical",
)

print(f"Answer : {output.answer}")
print(f"Status : {output.status}")
print(f"Factual: {output.confidence.factual:.3f}")
print(f"Reason : {output.confidence.reasoning:.3f}")
print(f"Boundary: {output.confidence.knowledge_boundary:.3f}")

## 5. Inspect the Null Set (What's Missing)

NSI detects concepts that were *expected* but *absent* from the input.

In [ ]:
for item in output.null_set.items:
    print(f"  Missing: {item.concept} "
          f"(type={item.null_type.name}, weight={item.weight:.2f})")

print(f"\nTotal anomaly score: {output.null_set.total_anomaly_score:.3f}")

## 6. Human-Readable Explanation

In [ ]:
explanation = model.explain(output)
print(explanation)

## Next Steps

- **02_symbolic_rules.ipynb** — Deep dive into rule definition, validation, and the prover.
- **03_metacognition.ipynb** — Explore confidence computation, self-doubt, and calibration.